In [ ]:
#Install Packages
#!pip install faiss-cpu

In [ ]:
#!pip install sentence-transformers

In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/codebasics/langchain/main/2_news_research_tool_project/notebooks/sample_text.csv')

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps cont...",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mu...,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity i...,Travel


### Step 1 : Create source embeddings for the text column

In [8]:
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer("all-mpnet-base-v2")

In [10]:
vectors = encoder.encode(df.text)

In [11]:
vectors

array([[-0.00247394,  0.03626724, -0.05290459, ..., -0.09152356,
        -0.0397    , -0.04330489],
       [-0.03357267,  0.00980519, -0.0325013 , ..., -0.05165467,
         0.0224589 , -0.03156179],
       [-0.01865323, -0.04051317, -0.01235389, ...,  0.00610587,
        -0.07179645,  0.02773852],
       ...,
       [-0.00066459,  0.04252123, -0.05645508, ...,  0.01315472,
        -0.03183569, -0.04357662],
       [-0.03317154,  0.03252453, -0.02484838, ...,  0.0117442 ,
         0.05747123,  0.0057102 ],
       [-0.00166395,  0.00413828, -0.04597083, ...,  0.02008528,
         0.05656245, -0.00161596]], dtype=float32)

In [12]:
vectors.shape

(8, 768)

In [14]:
vectors.shape[1]

768

In [13]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [15]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [16]:
index.add(vectors)

### Step 4 : Encode search text using same encorder and normalize the output vector

In [17]:
search_query = "I want to buy a polo t-shirt"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [18]:
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

### Step 5: Search for similar vector in the FAISS index created

In [20]:
index.search(svec, k=2)

(array([[1.3844838, 1.4039098]], dtype=float32), array([[3, 2]]))

In [21]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844838, 1.4039098]], dtype=float32)

In [22]:
I

array([[3, 2]])

In [23]:
df.iloc[I[0]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion
